In [2]:
import json
import pandas as pd
from src.utils.UsefulPaths import Paths
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

2023-09-03 00:03:38,926 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-03 00:03:39,067 - DEBUG - Creating converter from 7 to 5
2023-09-03 00:03:39,067 - DEBUG - Creating converter from 5 to 7
2023-09-03 00:03:39,067 - DEBUG - Creating converter from 7 to 5
2023-09-03 00:03:39,074 - DEBUG - Creating converter from 5 to 7


In [9]:
paths = Paths()

with open(paths.json_subsectors, 'r') as file:
    subsectors = json.load(file)

In [7]:
df_ipc_hypothesis = pd.read_csv(paths.ipc_subsetor_hypothesis)
df_ipc_hypothesis.head()

,IPC,title,zero_shot_hypothesis,similarities_hypothesis
0,G16Y,INFORMATION AND COMMUNICATION TECHNOLOGY SPECI...,"Other, Industry 4.0, Digital Media, Medical Te...","Other, Industry 4.0, Agriculture Technology, A..."
1,F24J,HEATING; RANGES; VENTILATING,"Medical Technology; Medical devices, Clean Tec...","Other, Medical Technology; Medical devices, Bi..."
2,F21H,INCANDESCENT MANTLES; OTHER INCANDESCENT BODIE...,"Other, Medical Technology; Medical devices, Bi...","Other, Educational Technology, Clean Technolog..."
3,G16B,"BIOINFORMATICS, i.e. INFORMATION AND COMMUNICA...","Other, Biopharmaceutical; Biotechonology, Indu...","Other, Biopharmaceutical; Biotechonology, Agri..."
4,A01P,"BIOCIDAL, PEST REPELLANT, PEST ATTRACTANT OR P...","Medical Technology; Medical devices, Biopharma...","Other, Agriculture Technology, Biopharmaceutic..."


In [4]:
ipc_dict = df_ipc_hypothesis.set_index('IPC').to_dict(orient='index')
for k, v in ipc_dict.items():
    ipc_dict[k]['zero_shot_hypothesis'] = ipc_dict[k]['zero_shot_hypothesis'].split(', ')
    ipc_dict[k]['similarities_hypothesis'] = ipc_dict[k]['similarities_hypothesis'].split(', ')

In [5]:
# # ABSTRACT BY JOAO
# df_abstract = pd.read_parquet('C:\\Users\\thiag\\PycharmProjects\\genome\\data\\processed\\abstract_and_classes.parquet')
# df_abstract.dropna(inplace=True)
# df_abstract.reset_index(drop=True, inplace=True)
# df_abstract

In [6]:
df_abstract = pd.read_parquet(paths.raw_parquet_abstract)
df_abstract.head()

,publication_number,abstract
0,20080063564,Embodiments of techniques for determining the ...
1,20080025285,A method for supporting frequency hopping of a...
2,20080056857,To correct any positional misalignment of a su...
3,20080031117,A holographic optical accessing system include...
4,20080056179,Transmitting an acknowledgement/negative ackno...


In [7]:
df_target = pd.read_csv(paths.processed_abstract_target)
df_target

,publication_number,subsector,name
0,20080027114,Agtech,leok
1,20080029392,Medtech / Medical devices,leok
2,20080012387,Advanced Manufacturing and Robotics,leok
3,20080047787,NaN,leok
4,20080013207,NaN,leok
...,...,...,...
195,20080033731,"Artificial Intelligence, Big Data and Analytics",joao
196,20080025538,Advanced Manufacturing and Robotics,joao
197,20080000184,Advanced Manufacturing and Robotics,joao
198,20080014753,Advanced Manufacturing and Robotics,joao


In [8]:
df = df_abstract[df_abstract['publication_number'].isin(df_target.publication_number)]
df

,publication_number,abstract
0,20080063564,Embodiments of techniques for determining the ...
1,20080025285,A method for supporting frequency hopping of a...
2,20080056857,To correct any positional misalignment of a su...
3,20080031117,A holographic optical accessing system include...
4,20080056179,Transmitting an acknowledgement/negative ackno...
...,...,...
245,20080014039,A retractable stanchion for supporting a semi-...
246,20080039760,"An adjustable adhesive bandage, comprising a f..."
247,20080018269,An electronic steering and control apparatus a...
248,20080004224,The invention provides methods for improving t...


In [9]:
selected_values = df_target.publication_number.tolist()
filtered_df  = df[df['publication_number'].isin(selected_values)]
df = filtered_df.set_index('publication_number').reindex(selected_values).reset_index()
df

,publication_number,abstract
0,20080027114,The invention relates to novel insecticidal ac...
1,20080029392,['The invention relates to a method of prepari...
2,20080012387,A vehicle support structure is provided with a...
3,20080047787,A disc brake for a vehicle such as a commercia...
4,20080013207,Embodiments of the present invention pertain t...
...,...,...
195,20080033731,Certain types of parametric spatial coding enc...
196,20080025538,Various embodiments are disclosed relating to ...
197,20080000184,"The invention is a flat-faced, or planar-clad,..."
198,20080014753,In a method of manufacturing a semiconductor d...


In [10]:
df['class_subsetor'] = df_target.subsector
df['name'] = df_target.name
df

,publication_number,abstract,class_subsetor,name
0,20080027114,The invention relates to novel insecticidal ac...,Agtech,leok
1,20080029392,['The invention relates to a method of prepari...,Medtech / Medical devices,leok
2,20080012387,A vehicle support structure is provided with a...,Advanced Manufacturing and Robotics,leok
3,20080047787,A disc brake for a vehicle such as a commercia...,NaN,leok
4,20080013207,Embodiments of the present invention pertain t...,NaN,leok
...,...,...,...,...
195,20080033731,Certain types of parametric spatial coding enc...,"Artificial Intelligence, Big Data and Analytics",joao
196,20080025538,Various embodiments are disclosed relating to ...,Advanced Manufacturing and Robotics,joao
197,20080000184,"The invention is a flat-faced, or planar-clad,...",Advanced Manufacturing and Robotics,joao
198,20080014753,In a method of manufacturing a semiconductor d...,Advanced Manufacturing and Robotics,joao


In [10]:
all_subsetor_names = []
all_subsetor_definitions = []
all_subsetor_keywords = []
all_subsetor_does_include = []
all_subsetor_does_not_include = []
for subsetor_name, subsetor_values in subsectors.items():
    subsetor_definition = subsetor_values.get('Definition', '')
    subsetor_keywords = subsetor_values.get('Keywords', '')
    subsetor_does_include = subsetor_values.get('Does include', '')
    subsetor_does_not_include = subsetor_values.get('Does not include', '')

    all_subsetor_names.append(subsetor_name)
    all_subsetor_definitions.append(subsetor_definition)
    all_subsetor_keywords.append(subsetor_keywords)
    all_subsetor_does_include.append(subsetor_does_include)
    all_subsetor_does_not_include.append(subsetor_does_not_include)

all_subsetor_names.append('Other')
all_subsetor_definitions.append('')
all_subsetor_keywords.append('')
all_subsetor_does_include.append('')
all_subsetor_does_not_include.append('')

In [31]:
classifier = pipeline("zero-shot-classification", model="sileod/deberta-v3-base-tasksource-nli")

# 'bert-base-uncased'
# 'bert-large-uncased-whole-word-masking'
# 'bert-base-multilingual-cased'
# 'all-mpnet-base-v2'
model = SentenceTransformer('all-mpnet-base-v2')
subsetor_definitions_embeddings = model.encode(all_subsetor_definitions, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)

break_line = 50

df['class_AI'] = ''
for index, row in df.iterrows():
    patent_abstract = row['abstract']

    query = f'What is the subsetor definiton that best describes this patent abstract "{patent_abstract}"'
    patent_abstract_embeddings = model.encode(query, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)

    cos_scores = util.cos_sim(patent_abstract_embeddings, subsetor_definitions_embeddings)

    p = pd.DataFrame(cos_scores.cpu().numpy().tolist(), columns=all_subsetor_names)
    v = p.iloc[0, :].sort_values(ascending=False)
    patent_similarities_hypothesis = v.index.tolist()
    patent_similarities_prob = v.values.tolist()

    patent_zero_shot_class = classifier(patent_abstract, all_subsetor_names, multi_label=False)
    patent_zero_shot_hypothesis = patent_zero_shot_class['labels']
    patent_zero_shot_prob = patent_zero_shot_class['scores']

    all_hypothesis = set()
    [all_hypothesis.add(label) for label in patent_zero_shot_class['labels'][0:3]]
    [all_hypothesis.add(label) for label in patent_similarities_hypothesis[0:3]]


    patent_zero_shot_class_2 = classifier(patent_abstract, list(all_hypothesis), multi_label=True)
    patent_zero_shot_multi_hypothesis = patent_zero_shot_class_2['labels']
    patent_zero_shot_multi_prob = patent_zero_shot_class_2['scores']

    df.at[index, 'class_AI'] = patent_zero_shot_class_2['labels'][0]
    # if index > break_line:
    #     break

2023-09-02 18:04:05,066 - DEBUG - https://huggingface.co:443 "HEAD /sileod/deberta-v3-base-tasksource-nli/resolve/main/config.json HTTP/1.1" 200 0
2023-09-02 18:04:05,956 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-09-02 18:04:06,447 - INFO - Use pytorch device: cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
df

,publication_number,abstract,class_subsetor,name,class_AI
0,20080027114,The invention relates to novel insecticidal ac...,Agtech,leok,Agriculture Technology
1,20080029392,['The invention relates to a method of prepari...,Medtech / Medical devices,leok,Medical Technology; Medical devices
2,20080012387,A vehicle support structure is provided with a...,Advanced Manufacturing and Robotics,leok,Industry 4.0
3,20080047787,A disc brake for a vehicle such as a commercia...,NaN,leok,Industry 4.0
4,20080013207,Embodiments of the present invention pertain t...,NaN,leok,Clean Technology
...,...,...,...,...,...
195,20080033731,Certain types of parametric spatial coding enc...,"Artificial Intelligence, Big Data and Analytics",joao,
196,20080025538,Various embodiments are disclosed relating to ...,Advanced Manufacturing and Robotics,joao,
197,20080000184,"The invention is a flat-faced, or planar-clad,...",Advanced Manufacturing and Robotics,joao,
198,20080014753,In a method of manufacturing a semiconductor d...,Advanced Manufacturing and Robotics,joao,


In [33]:
df.to_csv(paths.processed_abstract_result, index=False)
# df.to_excel('C:\\Users\\thiag\\PycharmProjects\\genome\\data\\processed\\abstract_result.xlsx', index=False)

In [ ]:
# classifier = pipeline("zero-shot-classification", model="sileod/deberta-v3-base-tasksource-nli")
#
# # 'bert-base-uncased'
# # 'bert-large-uncased-whole-word-masking'
# # 'bert-base-multilingual-cased'
# # 'all-mpnet-base-v2'
# model = SentenceTransformer('all-mpnet-base-v2')
# subsetor_definitions_embeddings = model.encode(all_subsetor_definitions, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)
#
# df_all = pd.DataFrame()
# for index, row in df_abstract.iterrows():
#     patent_abstract = row['abstract']
#
#     query = f'What is the subsetor definiton that best describes this patent abstract "{patent_abstract}"'
#     patent_abstract_embeddings = model.encode(query, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)
#
#     cos_scores = util.cos_sim(patent_abstract_embeddings, subsetor_definitions_embeddings)
#
#     p = pd.DataFrame(cos_scores.cpu().numpy().tolist(), columns=all_subsetor_names)
#     patent_similarities_hypothesis = p.iloc[0, :].to_frame().T.sort_values(by=0, axis=1, ascending=False).columns.tolist()
#
#     patent_zero_shot_class = classifier(patent_abstract, all_subsetor_names, multi_label=False)
#
#     p2 = pd.DataFrame([patent_zero_shot_class['scores']], columns=patent_zero_shot_class['labels'])
#     patent_zero_shot_hypothesis = ', '.join(patent_zero_shot_class['labels'])
#
#     patent_ipcs = row['class_concat'].split(', ')
#     all_hypothesis = set()
#     for patent_ipc in patent_ipcs:
#         ipc_zero_shot_hypothesis = ipc_dict[patent_ipc].get('zero_shot_hypothesis')
#         ipc_similarities_hypothesis = ipc_dict[patent_ipc].get('zero_shot_hypothesis')
#
#         candidate_zero_shot_label = ipc_zero_shot_hypothesis[0:5]
#         candidate_similarities_label = ipc_similarities_hypothesis[0:5]
#
#         output_zero_shot = classifier(patent_abstract, candidate_zero_shot_label, multi_label=False)
#         all_hypothesis.add(output_zero_shot['labels'][0])
#
#         output_similarities = classifier(patent_abstract, candidate_similarities_label, multi_label=False)
#         all_hypothesis.add(output_similarities['labels'][0])
#
#         a = 0
#
#     all_hypothesis.add(patent_zero_shot_class['labels'][0])
#     all_hypothesis.add(patent_similarities_hypothesis[0])
#     # [all_hypothesis.add(label) for label in patent_zero_shot_class['labels'][0:2]]
#     # [all_hypothesis.add(label) for label in patent_similarities_hypothesis[0:2]]
#     new_candidate_labels = list(all_hypothesis)
#     # new_candidate_labels.remove('Other')
#     output_similarities = classifier(patent_abstract, new_candidate_labels, multi_label=False)
#     a = 0

In [14]:
df_result_2 = pd.read_excel(paths.processed_abstract_result)
df_result_2

,publication_number,abstract,class_subsetor,class_AI,name
0,20080027114,The invention relates to novel insecticidal ac...,Agtech,Agriculture Technology,leok
1,20080029392,['The invention relates to a method of prepari...,Medtech / Medical devices,Medical Technology; Medical devices,leok
2,20080012387,A vehicle support structure is provided with a...,Advanced Manufacturing and Robotics,Industry 4.0,leok
3,20080047787,A disc brake for a vehicle such as a commercia...,NaN,Industry 4.0,leok
4,20080013207,Embodiments of the present invention pertain t...,NaN,Clean Technology,leok
...,...,...,...,...,...
195,20080033731,Certain types of parametric spatial coding enc...,"Artificial Intelligence, Big Data and Analytics",Digital Media,joao
196,20080025538,Various embodiments are disclosed relating to ...,Advanced Manufacturing and Robotics,Digital Media,joao
197,20080000184,"The invention is a flat-faced, or planar-clad,...",Advanced Manufacturing and Robotics,Advanced Manufacturing and Robotics,joao
198,20080014753,In a method of manufacturing a semiconductor d...,Advanced Manufacturing and Robotics,Clean Technology,joao


In [16]:
tp = 0
total = 0
for index, row in df_result_2.iterrows():
    old_name = subsectors[row['class_AI']].get('old_name')
    if row['class_subsetor'] == old_name:
        tp = tp + 1
    total = total + 1

In [17]:
precision = (tp / total) * 100